In [1]:
import argparse
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [3]:
def main():
    fix_seed = 2022
    random.seed(fix_seed)
    torch.manual_seed(fix_seed)
    np.random.seed(fix_seed)
    args = dotdict()
    # basic config
    args.model = 'LogSparse'
    args.plot_flat = 0
    args.verbose = 1
    args.is_training = 1
    args.inverse = False
    # data loader
    args.data = 'Wind'
    args.root_path = './dataset_example/WindData/dataset/'
    args.data_path ='wind_data.csv' 
    args.target = 'KVITEBJØRNFELTET'
    args.freq = '10min'
    args.checkpoints = './checkpoints/'
    args.checkpoint_flag = 1
    args.n_closet = None
    args.all_stations = 0
    args.data_step = 1
    args.min_num_nodes = 2
    # forecasting task
    args.features = 'M' #univariate
    args.seq_len = 96
    args.label_len = 48
    args.pred_len = 90
    args.enc_in = 1
    args.dec_in =1
    args.c_out = 1
    # model define
    args.d_model = 512
    args.n_heads = 8
    args.e_layers = 2
    args.d_layers = 1
    args.d_ff = 2048
    args.factor = 3
    args.distil = True
    args.dropout = 0.05
    args.embed = 'timeF'
    args.activation = 'gelu'
    args.output_attention = False
    args.win_len = 6
    args.res_len = None
    args.qk_ker = 4
    args.v_conv = 0
    args.sparse_flag = 1
    args.top_keys = 0
    args.kernel_size = 3
    args.train_strat_lstm = 'recursive'
    args.model_id= args.model + '_' + str(args.data) + '_' + str(args.pred_len)
    args.test_dir = ''
    # Optimization
    args.num_workers = 0
    args.itr = 1
    args.train_epochs = 10
    args.batch_size = 32
    args.patience= 5
    args.learning_rate = 0.001
    args.lr_decay_rate = 0.8
    args.des = 'test'
    args.loss = 'mse'
    args.lradj = 'type1'
    # GPU
    args.use_gpu = False 
    args.gpu = 0
    args.use_multi_gpu = False
    args.devices = '0,1,2,3'

    args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False
    if args.use_gpu and args.use_multi_gpu: 
        args.devices = args.devices.replace(' ', '')
        device_ids = args.devices.split(',')
        args.device_ids = [int(id_) for id_ in device_ids]
        args.gpu = args.device_ids[0]

    print('Args in experiment:')
    print(args)

    Exp = Exp_Main

    if args.is_training:
        for ii in range(args.itr):
            # setting record of experiments
            setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_{}'.format(
                args.model_id,
                args.model,
                args.data,
                args.features,
                args.seq_len,
                args.label_len,
                args.pred_len,
                ii)

            exp = Exp(args)  # set experiments
            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
            exp.train(setting)

            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.test(setting, base_dir=args.test_dir)

            torch.cuda.empty_cache()
    else:
        ii = 0
        setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting, test=1)
        torch.cuda.empty_cache()

In [4]:
main()

Args in experiment:
{'model': 'LogSparse', 'plot_flat': 0, 'verbose': 1, 'is_training': 1, 'inverse': False, 'data': 'Wind', 'root_path': './dataset_example/WindData/dataset/', 'data_path': 'wind_data.csv', 'target': 'KVITEBJØRNFELTET', 'freq': '10min', 'checkpoints': './checkpoints/', 'checkpoint_flag': 1, 'n_closet': None, 'all_stations': 0, 'data_step': 1, 'min_num_nodes': 2, 'features': 'M', 'seq_len': 96, 'label_len': 48, 'pred_len': 90, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'win_len': 6, 'res_len': None, 'qk_ker': 4, 'v_conv': 0, 'sparse_flag': 1, 'top_keys': 0, 'kernel_size': 3, 'train_strat_lstm': 'recursive', 'model_id': 'LogSparse_Wind_90', 'num_workers': 0, 'itr': 1, 'train_epochs': 10, 'batch_size': 32, 'patience': 5, 'learning_rate': 0.001, 'lr_decay_rate': 0.8, 'des': 'test', 'loss': 'mse

TypeError: object of type 'NoneType' has no len()